# Imports
Import python libraries

In [39]:
import os

# Set dataset parameters
Choose the attributes to load the dataset.

In [40]:


detectors = ['mtcnn_serfiq']  # ,'dlib', 'opencv', 'mediapipe','mtcnn','mtcnn_serfiq'
embedding_models = ['Facenet512','QMagFace','ArcFace']  # 'QMagFace', "ArcFace","Dlib", 'VGG-Face','FaceVACs', 'Facenet512','ArcFace_normalized'
quality_models = ['ser_fiq','tface'] #  'ser_fiq','tface'


# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [41]:
calibrators = ['logit']  # 'KDE', , 'isotonic','logit'
calibration_db = ['ForenFace'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW',

image_filters = ['roll']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion'
face_image_filters = []  # 'confusion_score'
quality_filters = [] # 'quality_group'

#all the filters in one dictionary
filters = {'image': image_filters, 'face_image': face_image_filters,'quality_image': quality_filters}

n_calibration_pairs = 10000



## Validation

In [42]:
enfsi_years = [2015]

# Video parameters

In [43]:
import numpy as np
#quality_dropout = [0.01, 0.02, 0.05, 0.10, 0.20, 0.5, 1]
quality_dropout = list(np.linspace(0.1,1.0,10))

## Other parameters
Rest of the configuration

In [44]:
metrics = 'euclidean_l2'
embedding_model_as_scorer = True

# Scenarios
Different filters to do computations

In [45]:
scenario0 = []
scenario1 = ['gender', 'age']
scenario2 = ['yaw', 'pitch']
scenario3 = ['glasses', 'beard']
scenario4 = ['low_quality']
scenario5 = ['headgear']
scenario6 = ['terhorst_quality_rounded']

# Input and output directories
Where the sql database is and which folder the results are saved.

In [46]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'output_3')

# Preprocess data
Connect to sql database.

In [47]:
from sql_face.alchemy import get_session
from sql_face.tables import *

In [48]:
session = get_session(input_dir, db_name)

# Experiments

All parameters are specified in ExperimentalSetup, that creates a different experiment with all the posible combinations of detectors, embedding_models, calibrators, etc.

In [49]:
from lr_video_face.experiments import ExperimentalSetup

In [50]:
experimental_setup = ExperimentalSetup(detectors, 
                                        embedding_models, 
                                        quality_models,
                                        calibrators, 
                                        calibration_db, 
                                        enfsi_years,
                                        filters,
                                        metrics, 
                                        n_calibration_pairs,
                                        embedding_model_as_scorer,
                                        output_dir,
                                        session,
                                        quality_dropout)

# Results

In [51]:
""" from sql_face.tables import Image, Quality
query = session.query(Image).filter(Image.image_quality == Quality.MEDIUM).all()

a = len(query)
print (a) """


' from sql_face.tables import Image, Quality\nquery = session.query(Image).filter(Image.image_quality == Quality.MEDIUM).all()\n\na = len(query)\nprint (a) '

In [52]:
'''for exp in experimental_setup:
    exp.perform()
    break  '''

'for exp in experimental_setup:\n    exp.perform()\n    break  '

In [53]:
from lr_video_face.evaluators import GlobalEvaluator  

In [54]:
global_results = GlobalEvaluator(experimental_setup)    

100%|██████████| 6/6 [05:40<00:00, 56.69s/it]


## Individual plots

In [55]:
global_results.make_experiment_plots()

## General plot

In [56]:
# import pickle
# with open('data2.db','wb') as outp:
#     pickle.dump(global_results,outp,-1)

In [57]:
global_results.make_global_plot()

0.7464577308658773 [0.22098116 0.44689484 0.94679573]
0.8893585694545896 [0.32713992 0.63820658 1.15104175]
0.8213780092409189 [0.27047181 0.56390878 1.04556149]
1.0035942997287028 [0.40065865 0.77156587 1.35836176]
0.9507670741658851 [0.3725784  0.72073686 1.2641242 ]
1.0577047494893814 [0.41605722 0.8223408  1.48217711]
1.120024385466074 [0.42527955 0.86256999 1.63488047]
1.1695374433722718 [0.4343456  0.91173092 1.78086573]
1.2075554941589037 [0.44503396 0.96037959 1.87492617]
1.244801425788463 [0.45318185 1.01601481 1.98477709]
1.0152976893742889 [0.23900159 0.55471404 1.41620847]
1.1079577381702541 [0.27940019 0.70390593 1.6083688 ]
1.0612708924492065 [0.25525495 0.60688619 1.48215494]
1.2432589391550537 [0.32381712 0.87875335 1.89711364]
1.1868894383049906 [0.29924016 0.80766925 1.74532315]
1.2715311775588152 [0.33843686 0.93304568 2.00774067]
1.289054039031904 [0.34380806 0.94241792 2.07278139]
1.3219057585223024 [0.35338589 0.97542655 2.16613136]
1.3452779789694562 [0.36732129 